# Rebuild All Events
    This script builds/rebuilds all the events of Live Mouse Tracker
    You can also create your own events and add them to this process
    
    Approximate time is 1h of computation for a 24h database of 4 animals


# Parameters:
    This will compute automatically events for your database, and adjust settings for you.
    
    If you remove the automatic settings, you can force parameters:
        Set minT and maxT to process the database.
        Set windowT to divide the computation load in segment of that duration. Default value is 1 day.

In [1]:
import sys
sys.path.insert(1, "../")
from lmtanalysis.Measure import *

# set this to false if you want to set manual parameters.
automaticSettings = True

# Manual parameters:

''' minT and maxT to process the analysis (expressed in frame) '''
minT = 0
#maxT = 2*oneDay
maxT = input("Enter maximal time window to compute the event in frames. You can also set in days, hour, minutes. Valid entries include: 100, 1d, 1.5d, 23.5h, 1d 2h 3m 4s 5f ")
''' time window to compute the events (in frame). '''
windowT = 1*oneDay
''' speed up process '''
USE_CACHE_LOAD_DETECTION_CACHE = True

Enter maximal time window to compute the event in frames. You can also set in days, hour, minutes. Valid entries include: 100, 1d, 1.5d, 23.5h, 1d 2h 3m 4s 5f 2d


# Run this section to compute your database
    You will be prompt to provide a database

In [2]:
''' Created on 26 march 2019 @author: Fab '''

import sys
sys.path.insert(1, "../")
import sqlite3
from lmtanalysis.Animal import *
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis import BuildEventTrain3, BuildEventTrain4, BuildEventTrain2, BuildEventFollowZone, BuildEventRear5, BuildEventFloorSniffing,\
    BuildEventSocialApproach, BuildEventSocialEscape, BuildEventApproachContact,BuildEventOralOralContact,\
    BuildEventApproachRear, BuildEventGroup2, BuildEventGroup3, BuildEventGroup4, BuildEventOralGenitalContact, \
    BuildEventStop, BuildEventWaterPoint, \
    BuildEventMove, BuildEventGroup3MakeBreak, BuildEventGroup4MakeBreak,\
    BuildEventSideBySide, BuildEventSideBySideOpposite, BuildEventDetection,\
    BuildDataBaseIndex, BuildEventWallJump, BuildEventSAP,\
    BuildEventOralSideSequence, CheckWrongAnimal,\
    CorrectDetectionIntegrity, BuildEventNest4, BuildEventNest3, BuildEventGetAway
    
from psutil import virtual_memory

from tkinter.filedialog import askopenfilename
from lmtanalysis.TaskLogger import TaskLogger
import sys
import traceback
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.EventTimeLineCache import flushEventTimeLineCache,\
    disableEventTimeLineCache


from lmtanalysis.EventTimeLineCache import EventTimeLineCached

def getNumberOfFrames(file):
    
    connection = sqlite3.connect( file )
    c = connection.cursor() 
    query = "SELECT MAX(FRAMENUMBER) FROM FRAME";
    c.execute( query )
    result = c.fetchall()
    numberOfFrames = int( result[0][0] )
    connection.close()
    
    return numberOfFrames

class FileProcessException(Exception):
    pass

eventClassList = [                  
                  BuildEventOralOralContact,
                  BuildEventOralGenitalContact,
                  BuildEventSideBySide,
                  BuildEventSideBySideOpposite,
                  BuildEventTrain2,                  
                  BuildEventTrain3,
                  BuildEventTrain4,
                  BuildEventMove,
                  BuildEventFollowZone,
                  BuildEventRear5,
                  BuildEventSocialApproach,
                  BuildEventGetAway,
                  BuildEventSocialEscape,
                  BuildEventApproachRear,
                  BuildEventGroup2,
                  BuildEventGroup3,
                  BuildEventGroup4,
                  BuildEventGroup3MakeBreak,
                  BuildEventGroup4MakeBreak,
                  BuildEventStop,
                  BuildEventWaterPoint,
                  BuildEventApproachContact,
                  BuildEventWallJump,
                  BuildEventSAP,
                  BuildEventOralSideSequence,
                  BuildEventNest3,
                  BuildEventNest4
                   ]

def flushEvents( connection ):
    
    print("Flushing events...")

    for ev in eventClassList:
        
        chrono = Chronometer( "Flushing event " + str(ev) )
        ev.flush( connection );      
        chrono.printTimeInS()
    

def processTimeWindow( connection, currentMinT , currentMaxT ):
    
    CheckWrongAnimal.check( connection, tmin=currentMinT, tmax=currentMaxT )
    
    # Warning: enabling this process (CorrectDetectionIntegrity) will alter the database permanently
    # CorrectDetectionIntegrity.correct( connection, tmin=0, tmax=maxT )
                            
    BuildEventDetection.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT )

    animalPool = None
    
    flushEventTimeLineCache()
    
    if ( USE_CACHE_LOAD_DETECTION_CACHE ):
        print("Caching load of animal detection...")
        animalPool = AnimalPool( )
        animalPool.loadAnimals( connection )
        animalPool.loadDetection( start = currentMinT, end = currentMaxT )
        print("Caching load of animal detection done.")

    for ev in eventClassList:
        
        chrono = Chronometer( str( ev ) )
        ev.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT, pool = animalPool )        
        chrono.printTimeInS()

def process( file ):

    print(file)
    
    if automaticSettings:
        print("Automatic settings.")
        windowT = 1*oneDay
        minT = 0
        maxT = getNumberOfFrames( file )
        print ( "Auto max set to" , maxT , "frames")
        
    
    chronoFullFile = Chronometer("File " + file )
    
    connection = sqlite3.connect( file )
    
    # update missing fields
    try:
        connection = sqlite3.connect( file )    
        c = connection.cursor()            
        query = "ALTER TABLE EVENT ADD METADATA TEXT";
        c.execute( query )    
        connection.commit()
        c.close()
        #connection.close()
    except:
        print( "METADATA field already exists" , file )
        
    #build index
    
    BuildDataBaseIndex.buildDataBaseIndex( connection, force=False )
        
    '''
    # build sensor data    
    animalPool = AnimalPool( )
    animalPool.loadAnimals( connection )
    animalPool.buildSensorData(file)
    '''
    
    currentT = minT

    try:

        flushEvents( connection )
        
        while currentT < maxT:
                        
            currentMinT = currentT
            currentMaxT = currentT+ windowT
            if ( currentMaxT > maxT ):
                currentMaxT = maxT
                
            chronoTimeWindowFile = Chronometer("File "+ file+ " currentMinT: "+ str(currentMinT)+ " currentMaxT: " + str(currentMaxT) );
            processTimeWindow( connection, currentMinT, currentMaxT )    
            chronoTimeWindowFile.printTimeInS()
            
            currentT += windowT

                        

        print("Full file process time: ")
        chronoFullFile.printTimeInS()
        
        TEST_WINDOWING_COMPUTATION = False
        
        if ( TEST_WINDOWING_COMPUTATION ):
                
            print("*************")
            print("************* TEST START SECTION")
            print("************* Test if results are the same with or without the windowing.")
            
            # display and record to a file all events found, checking with rolling idA from None to 4. Save nbEvent and total len
            
            eventTimeLineList = []
            
            eventList = getAllEvents( connection )
            file = open("outEvent"+str(windowT)+".txt","w")  
            file.write( "Event name\nnb event\ntotal duration" )
            
            for eventName in eventList:
                for idAnimalA in range( 0,5 ):                
                        idA = idAnimalA 
                        if idA == 0:
                            idA = None
                        timeLine = EventTimeLineCached( connection, file, eventName, idA,  minFrame=minT, maxFrame=maxT )
                        eventTimeLineList.append( timeLine )
                        file.write( timeLine.eventNameWithId+"\t"+str(len(timeLine.eventList))+"\t"+str(timeLine.getTotalLength())+"\n" )            
            
            file.close() 
    
            #plotMultipleTimeLine( eventTimeLineList )
            
            print("************* END TEST")
        
        
    except:
        
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        error = ''.join('!! ' + line for line in lines)
        
        t = TaskLogger( connection )
        t.addLog( error )
        
        print( error, file=sys.stderr ) 
        
        raise FileProcessException()
            
def getAllEvents( connection ):
    
    query = "select name from event group by name order by name"
    c = connection.cursor()     
    c.execute( query )    
    all_rows = c.fetchall()        
    header = [ "Name" ]
    data =[]    
    for row in all_rows:
        data.append( row[0] )
    return data        

print("Code launched.")

mem = virtual_memory()
availableMemoryGB = mem.total / 1000000000
print( "Total memory on computer: (GB)", availableMemoryGB ) 

if availableMemoryGB < 10:
    print( "Not enough memory to use cache load of events.")
    disableEventTimeLineCache()

print("A window is popping (maybe hidden) asking for files to process...")
files = getFilesToProcess()

chronoFullBatch = Chronometer("Full batch" )    

if ( files != None ):

    for file in files:
        try:
            print ( "Processing file" , file )
            process( file )
        except FileProcessException:
            print ( "STOP PROCESSING FILE " + file , file=sys.stderr  )

        flushEventTimeLineCache()

chronoFullBatch.printTimeInS()
print( "*** ALL JOBS DONE ***")

Code launched.
Total memory on computer: (GB) 17.095544832
A window is popping (maybe hidden) asking for files to process...
Processing file D:/live mouse tracker data/Experiment 5004ter/Experiment 5004ter.sqlite
D:/live mouse tracker data/Experiment 5004ter/Experiment 5004ter.sqlite
Automatic settings.
Auto max set to 4553416 frames
METADATA field already exists D:/live mouse tracker data/Experiment 5004ter/Experiment 5004ter.sqlite
Creating lmtanalysis indexes...
CREATE INDEX `animalIndex` ON `ANIMAL` (`ID` );
index animalIndex already exists
CREATE INDEX `detectionIndex` ON `DETECTION` (`ID` ASC,`FRAMENUMBER` ASC);
index detectionIndex already exists
CREATE INDEX `detetIdIndex` ON `DETECTION` (`ID` ASC);
index detetIdIndex already exists
CREATE INDEX `detframenumberIndex` ON `DETECTION` (`FRAMENUMBER` ASC);
CREATE INDEX `eventEndFrameIndex` ON `EVENT` (`ENDFRAME` ASC);
CREATE INDEX `eventIndex` ON `EVENT` (`ID` ASC,`STARTFRAME` ASC,`ENDFRAME` ASC);
index eventIndex already exists
CR

Number of event deleted: 0 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventStop' from '..\\lmtanalysis\\BuildEventStop.py'>  ]  1.0092277526855469  seconds
Deleting event Water Zone (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Water Zone"
Number of event deleted: 24284 
Deleting event Water Stop (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Water Stop"
Number of event deleted: 185 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventWaterPoint' from '..\\lmtanalysis\\BuildEventWaterPoint.py'>  ]  19.599931955337524  seconds
Deleting event Approach contact (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Approach contact"
Number of event deleted: 0 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventApproachContact' from '..\\lmtanalysis\\BuildEventApproachContact.py'>  ]  0.4981207847595215  seconds
Deleting event WallJump (idA:None,idB:None,idC:None,idD:None) from base

Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:1 idC:None idD:None loadEvent: False
Number of event:  9324
Mean length of event:  5.572608322608323
first event frame:  1216
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 9324 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:3 idC:None idD:None loadEvent: False
Number of event:  7522
Mean length of event:  6.455862802446158
first event frame:  2645
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 7522 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:4 idC:None idD:None loadEvent: False
Number of event:  7351
Mean length of event:  6.743980410828459
first event frame:  883
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 7351 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:3 idB:1 idC:None idD:None

Number of event:  7645
Mean length of event:  9.494833224329627
first event frame:  190
Keep previous entry.
Saving timeLine: Side by side Contact ( 7645 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:2 idB:4 idC:None idD:None loadEvent: False
Number of event:  7774
Mean length of event:  9.181116542320556
first event frame:  1629
Keep previous entry.
Saving timeLine: Side by side Contact ( 7774 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:3 idB:1 idC:None idD:None loadEvent: False
Number of event:  9172
Mean length of event:  10.781508940252944
first event frame:  997
Keep previous entry.
Saving timeLine: Side by side Contact ( 9172 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:3 idB:2 idC:None idD:None loadEvent: False
Number of event:  7645
Mean length of event:  9.494833224329627
first event fram

Oral-genital Contact  Id( 3 , 1 , None , None ) Min/maxFrame: ( 0 / 2592000 ) Loaded ( 9505  records loaded in  2.004452705383301 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 2 , None , None ) Min/maxFrame: ( 0 / 2592000 ) Loaded ( 7846  records loaded in  1.9914493560791016 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 4 , None , None ) Min/maxFrame: ( 0 / 2592000 ) Loaded ( 7321  records loaded in  1.9909517765045166 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 1 , None , None ) Min/maxFrame: ( 0 / 2592000 ) Loaded ( 8494  records loaded in  1.9914498329162598 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 2 , None , None ) Min/maxFrame: ( 0 / 2592000 ) Loaded ( 9506  records loaded in  2.0014524459838867 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 3 , None , None ) Min/maxFrame: ( 0 / 2592000 ) Loaded ( 7411  records loaded in  1.9899532794952393 S )
Caching eventTimeLine
Train2
Event Train2 created. eventNameWithId = Train2 idA:1

...

Social approach  Id( 1 , 4 , None , None ) Loaded from cache ( 22431  records. )
Contact  Id( 2 , 1 , None , None ) Loaded from cache ( 2874  records. )
Social approach  Id( 2 , 1 , None , None ) Loaded from cache ( 29317  records. )
Contact  Id( 2 , 3 , None , None ) Loaded from cache ( 3349  records. )
Social approach  Id( 2 , 3 , None , None ) Loaded from cache ( 26267  records. )
Contact  Id( 2 , 4 , None , None ) Loaded from cache ( 3220  records. )
Social approach  Id( 2 , 4 , None , None ) Loaded from cache ( 24063  records. )
Contact  Id( 3 , 1 , None , None ) Loaded from cache ( 2948  records. )
Social approach  Id( 3 , 1 , None , None ) Loaded from cache ( 28047  records. )
Contact  Id( 3 , 2 , None , None ) Loaded from cache ( 3349  records. )
Social approach  Id( 3 , 2 , None , None ) Loaded from cache ( 24256  records. )
Contact  Id( 3 , 4 , None , None ) Loaded from cache ( 2334  records. )
Social approach  Id( 3 , 4 , None , None ) Loaded from cache ( 26517  records. )
C

mean animal bodysize:  22.2189648106572
Oral-oral Contact  Id( 4 , 1 , None , None ) Min/maxFrame: ( 2592000 / 4553416 ) Loaded ( 3638  records loaded in  7.162627696990967 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 1 , None , None ) Loaded from cache ( 3892  records. )
Oral-oral Contact  Id( 2 , 1 , None , None ) Min/maxFrame: ( 2592000 / 4553416 ) Loaded ( 5118  records loaded in  7.15922737121582 S )
Caching eventTimeLine
Oral-genital Contact  Id( 2 , 1 , None , None ) Loaded from cache ( 5044  records. )
Oral-oral Contact  Id( 3 , 1 , None , None ) Min/maxFrame: ( 2592000 / 4553416 ) Loaded ( 3239  records loaded in  7.1546947956085205 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 1 , None , None ) Loaded from cache ( 3899  records. )
Animal Id:4 Name:D RFID:000022986503 Genotype:KO User1:None
mean animal bodysize:  20.986850295000608
Oral-oral Contact  Id( 1 , 4 , None , None ) Min/maxFrame: ( 2592000 / 4553416 ) Loaded ( 3638  records loaded in  7.153615713

Number of event:  181
Mean length of event:  74.1878453038674
first event frame:  2661629
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 181 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:1 idB:4 idC:None idD:None loadEvent: False
seq oral geni - oral oral idA:1 idB:4 idC:None idD:None
Number of event:  210
Mean length of event:  65.18571428571428
first event frame:  2667728
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 210 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:2 idB:1 idC:None idD:None loadEvent: False
seq oral geni - oral oral idA:2 idB:1 idC:None idD:None
Number of event:  317
Mean length of event:  61.46372239747634
first event frame:  2661454
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 317 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:2 idB:3 idC:None idD:None 

Number of event:  661
Mean length of event:  92.93343419062028
first event frame:  2654993
Keep previous entry.
Saving timeLine: Nest3_ ( 661 events )
Number of event:  1977
Mean length of event:  79.34800202326758
first event frame:  2659385
Keep previous entry.
Saving timeLine: Nest3_ ( 1977 events )
Number of event:  3120
Mean length of event:  84.5048076923077
first event frame:  2651215
Keep previous entry.
Saving timeLine: Nest3_ ( 3120 events )
Number of event:  1190
Mean length of event:  78.98991596638655
first event frame:  2659360
Keep previous entry.
Saving timeLine: Nest3_ ( 1190 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Nest3','0','2020-06-05 01:38:10','2592000','4553416' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventNest3' from '..\\lmtanalysis\\BuildEventNest3.py'>  ]  85.42885828018188  seconds
[NEST 4] : Assume that there is no occlusion
Contact  Id( 1 , 2 , None , None ) Loaded from cache ( 2874  records. )
C

INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Nest4','0','2020-06-05 01:38:54','2592000','4553416' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventNest4' from '..\\lmtanalysis\\BuildEventNest4.py'>  ]  43.69185781478882  seconds
[Chrono  File D:/live mouse tracker data/Experiment 5004ter/Experiment 5004ter.sqlite currentMinT: 2592000 currentMaxT: 4553416  ]  4507.639219284058  seconds
Full file process time: 
[Chrono  File D:/live mouse tracker data/Experiment 5004ter/Experiment 5004ter.sqlite  ]  20248.36396241188  seconds
[Chrono  Full batch  ]  20249.012149572372  seconds
*** ALL JOBS DONE ***
